In [1]:
num_sample = 300
num_burn = 20
sample_size = 800

In [2]:
from dgp import *

# 1. Simulate or load data
min_degree = 1
max_degree = 4
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:10<00:00, 31.40it/s]


In [3]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.302375,
 'direct_effect': -0.14652500000000002,
 'spillover_effect': -0.06694999999999984,
 'psi_1_gamma': 0.2579000000000001,
 'psi_0_gamma': 0.4044250000000001,
 'psi_zero': 0.47137499999999993}

In [4]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.1419511419307552, 0.18673749999999997)

In [5]:
n_cpu = 10

In [6]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x1134a0310> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [7]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.31531 ± 0.05474
direct_effect: -0.13586 ± 0.07862
spillover_effect: -0.06052 ± 0.06265
psi_0_gamma: 0.40984 ± 0.05887
psi_zero: 0.47035 ± 0.02341
psi_1_gamma: 0.27398 ± 0.06697


In [8]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x168f00ee0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [9]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.31235 ± 0.03757
direct_effect: -0.13121 ± 0.03759
spillover_effect: -0.06244 ± 0.04131
psi_0_gamma: 0.40777 ± 0.03702
psi_zero: 0.47021 ± 0.02322
psi_1_gamma: 0.27656 ± 0.04194


In [10]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr_em at 0x168f00e50> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.31137 ± 0.05317
direct_effect: -0.13487 ± 0.07812
spillover_effect: -0.06368 ± 0.06167
psi_0_gamma: 0.40551 ± 0.05964
psi_zero: 0.46919 ± 0.02494
psi_1_gamma: 0.27064 ± 0.06766


In [11]:
from dgp import *

# 1. Simulate or load data
min_degree = 1
max_degree = 4
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain2(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:11<00:00, 27.58it/s]


In [12]:
from agcEffect import *

agc_effect2(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.30519999999999997,
 'direct_effect': -0.08134999999999987,
 'spillover_effect': -0.05460000000000015,
 'psi_1_gamma': 0.28437500000000004,
 'psi_0_gamma': 0.3657249999999999,
 'psi_zero': 0.42032500000000006}

In [13]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.10774721528057304, 0.18673749999999997)

In [14]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x1134a0310> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [15]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.30471 ± 0.05112
direct_effect: -0.06150 ± 0.08528
spillover_effect: -0.07708 ± 0.05273
psi_0_gamma: 0.34703 ± 0.05178
psi_zero: 0.42411 ± 0.02234
psi_1_gamma: 0.28552 ± 0.06831


In [16]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x168f00ee0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [17]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.28514 ± 0.03317
direct_effect: -0.09657 ± 0.03993
spillover_effect: -0.06310 ± 0.03839
psi_0_gamma: 0.35635 ± 0.03598
psi_zero: 0.41945 ± 0.02122
psi_1_gamma: 0.25978 ± 0.03858


In [18]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr_em at 0x168f00e50> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.29025 ± 0.05149
direct_effect: -0.07452 ± 0.07347
spillover_effect: -0.07990 ± 0.05767
psi_0_gamma: 0.34401 ± 0.05534
psi_zero: 0.42391 ± 0.02349
psi_1_gamma: 0.26949 ± 0.06662


In [19]:
from dgp import *

# 1. Simulate or load data
min_degree = 3
max_degree = 7
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:10<00:00, 31.34it/s]


In [20]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.21535000000000004,
 'direct_effect': -0.13037499999999994,
 'spillover_effect': -0.27842500000000014,
 'psi_1_gamma': 0.175575,
 'psi_0_gamma': 0.30594999999999994,
 'psi_zero': 0.5843750000000001}

In [21]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.15146335014349138, 0.22922499999999998)

In [22]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x1134a0310> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [23]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.24407 ± 0.06616
direct_effect: -0.10483 ± 0.09987
spillover_effect: -0.22072 ± 0.10387
psi_0_gamma: 0.31891 ± 0.08362
psi_zero: 0.53963 ± 0.05288
psi_1_gamma: 0.21408 ± 0.07891


In [24]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x168f00ee0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [25]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.25756 ± 0.03965
direct_effect: -0.11716 ± 0.03582
spillover_effect: -0.18727 ± 0.06593
psi_0_gamma: 0.34404 ± 0.04654
psi_zero: 0.53131 ± 0.03638
psi_1_gamma: 0.22687 ± 0.04017


In [26]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr_em at 0x168f00e50> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.25473 ± 0.06058
direct_effect: -0.11809 ± 0.09123
spillover_effect: -0.20176 ± 0.09247
psi_0_gamma: 0.33636 ± 0.06953
psi_zero: 0.53812 ± 0.05675
psi_1_gamma: 0.21826 ± 0.07878


In [27]:
from dgp import *

# 1. Simulate or load data
min_degree = 3
max_degree = 7
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain2(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:12<00:00, 25.76it/s]


In [28]:
from agcEffect import *

agc_effect2(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.20065000000000002,
 'direct_effect': 0.019100000000000034,
 'spillover_effect': -0.22932500000000008,
 'psi_1_gamma': 0.2101,
 'psi_0_gamma': 0.19099999999999998,
 'psi_zero': 0.42032500000000006}

In [29]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.07165969958346469, 0.22922499999999998)

In [30]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x1134a0310> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [31]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.17747 ± 0.06160
direct_effect: -0.00865 ± 0.08826
spillover_effect: -0.23985 ± 0.08447
psi_0_gamma: 0.18353 ± 0.06636
psi_zero: 0.42338 ± 0.04567
psi_1_gamma: 0.17488 ± 0.07852


In [32]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x168f00ee0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [33]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.16397 ± 0.03051
direct_effect: -0.03655 ± 0.02509
spillover_effect: -0.21508 ± 0.05354
psi_0_gamma: 0.19252 ± 0.03295
psi_zero: 0.40760 ± 0.03208
psi_1_gamma: 0.15597 ± 0.03303


In [34]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr_em at 0x168f00e50> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.18051 ± 0.05565
direct_effect: -0.00248 ± 0.08375
spillover_effect: -0.23678 ± 0.08272
psi_0_gamma: 0.18536 ± 0.05812
psi_zero: 0.42214 ± 0.04995
psi_1_gamma: 0.18288 ± 0.07547


In [35]:
from dgp import *

# 1. Simulate or load data
min_degree = 4
max_degree = 10
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:10<00:00, 29.52it/s]


In [36]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.17247500000000002,
 'direct_effect': -0.11772500000000002,
 'spillover_effect': -0.40012499999999995,
 'psi_1_gamma': 0.13674999999999998,
 'psi_0_gamma': 0.254475,
 'psi_zero': 0.6546}

In [37]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.1595891533015739, 0.2709375)

In [38]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x1134a0310> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [39]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.22475 ± 0.05756
direct_effect: -0.08846 ± 0.09269
spillover_effect: -0.27191 ± 0.13148
psi_0_gamma: 0.29052 ± 0.07458
psi_zero: 0.56243 ± 0.10574
psi_1_gamma: 0.20206 ± 0.07084


In [40]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x168f00ee0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [41]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.22762 ± 0.04036
direct_effect: -0.11200 ± 0.03362
spillover_effect: -0.25423 ± 0.08455
psi_0_gamma: 0.31099 ± 0.05131
psi_zero: 0.56522 ± 0.04676
psi_1_gamma: 0.19899 ± 0.03896


In [42]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr_em at 0x168f00e50> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.23603 ± 0.06889
direct_effect: -0.07813 ± 0.10063
spillover_effect: -0.26342 ± 0.15352
psi_0_gamma: 0.29088 ± 0.07686
psi_zero: 0.55430 ± 0.13864
psi_1_gamma: 0.21276 ± 0.08844


In [ ]:
from dgp import *

# 1. Simulate or load data
min_degree = 4
max_degree = 10
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain2(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0)

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

100%|██████████| 320/320 [00:11<00:00, 26.75it/s]


In [44]:
from agcEffect import *

agc_effect2(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=50,
    burnin_R=5,
    seed=0
)

{'average': 0.08635,
 'direct_effect': 0.04445,
 'spillover_effect': -0.36242500000000005,
 'psi_1_gamma': 0.10235000000000001,
 'psi_0_gamma': 0.05790000000000001,
 'psi_zero': 0.42032500000000006}

In [45]:
dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

(-0.028277580600158437, 0.3198625)

In [46]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr at 0x1134a0310> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [47]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

average: 0.06505 ± 0.02068
direct_effect: 0.00630 ± 0.02311
spillover_effect: -0.34308 ± 0.13873
psi_0_gamma: 0.06120 ± 0.01881
psi_zero: 0.40428 ± 0.13529
psi_1_gamma: 0.06750 ± 0.02509


In [48]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)

Multiprocessing <function run_autognet at 0x168f00ee0> in 100 tasks, with 10 processes...
Multiprocessing finished.


In [49]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.06390 ± 0.01846
direct_effect: 0.00374 ± 0.01354
spillover_effect: -0.31510 ± 0.06821
psi_0_gamma: 0.06251 ± 0.01690
psi_zero: 0.37761 ± 0.06109
psi_1_gamma: 0.06625 ± 0.02152


In [50]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr_em at 0x168f00e50> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.07516 ± 0.03175
direct_effect: 0.04005 ± 0.05249
spillover_effect: -0.35543 ± 0.12202
psi_0_gamma: 0.04705 ± 0.03331
psi_zero: 0.40248 ± 0.11483
psi_1_gamma: 0.08709 ± 0.04088
